In [1]:
import pandas as pd
from IPython.display import display
from sklearn import preprocessing
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.python.saved_model.simple_save import simple_save

data=pd.read_csv("../Data/creditcard.csv")
del data['Time']


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

In [4]:
#separate X and Y
X=data.iloc[:,0:-1].values
Y=data.iloc[:,-1].values

#Normalize data
min_max_scaler=preprocessing.MinMaxScaler()
X=min_max_scaler.fit_transform(X)

#Separate train/Test data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=230)


learning_rate=0.02
epochs=1
batch_size=30

x=tf.placeholder(tf.float32,[None,29])
y=tf.placeholder(tf.float32,[None,1])

#declare weight connecting input layer to hidden layer
W1=tf.Variable(tf.random_normal([29,6]),name='W1')
b1=tf.Variable(tf.random_normal([6]),name='b1')

#declare weights connecting the hidden layer to the output layer
W2=tf.Variable(tf.random_normal([6,1]),name='W2')
b2=tf.Variable(tf.random_normal([1]),name='b2')

#calculate the output of the hidden layer
hidden_out=tf.add(tf.matmul(x,W1),b1)
hidden_out=tf.nn.relu(hidden_out)

#calculate the hidden layer output, using softmax
y_=tf.nn.sigmoid(tf.add(tf.matmul(hidden_out,W2),b2))

y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped)+ (1 - y) * tf.log(1 - y_clipped), axis=1))

# add an optimiser
optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)


# finally setup the initialisation operator
init_op = tf.global_variables_initializer()

# define an accuracy assessment operation
prediction=tf.round(y_)
correct_prediction = tf.equal(tf.round(y), tf.round(y_))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#acc, acc_op = tf.metrics.accuracy(labels=tf.argmax(labels, 1), predictions=tf.argmax(logits,1))

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# start the session
with tf.Session() as sess:
    # initialise the variables
    sess.run(init_op)
    total_batch = int(X_train.shape[0]/ batch_size)
    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            #batch_x=np.array([X_train[i]]);
            #batch_y=np.matrix([y_train[i]]);
            if i==0:
                batch_x=X_train[0:batch_size*(i+1),:];
                batch_y=np.transpose([y_train[0:batch_size*(i+1)]])
            else:
                batch_x=X_train[batch_size*i:batch_size*(i+1),:];
                batch_y=np.transpose([y_train[batch_size*i:batch_size*(i+1)]])
            _, c = sess.run([optimiser, cross_entropy], feed_dict={x: batch_x, y: batch_y})
            avg_cost += c / total_batch
        print ("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost))
        print (sess.run(accuracy, feed_dict={x: X_test, y: np.transpose(np.matrix(y_test))}))
        #output= sess.run(prediction,feed_dict={x: batch_x, y: (np.matrix(batch_y))})
    simple_save(sess, "../SavedModels/TFModel/", inputs={"inputs": x}, outputs={"outputs": y})
    print ("done")



Epoch: 1 cost = 0.013
0.9982269
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ../SavedModels/TFModel/saved_model.pb
done
